# Benchmark base64 algos

A naive base64 algorithm is 30 times slower inside cling that with a standard exe.

Using a fast base64 algorithm, like https://github.com/aklomp/base64 could improve the situation (it is 5 times faster than the naive implementation below in base64_naive.cpp), but the main reason of base64 being slow is not here; it could be 30 times faster.


In [1]:
#define NO_MAIN
#include "base64_naive.cpp"

#include "xtl/xbase64.hpp"

In [2]:
auto buf = read_binary_file("data/lena.png");

In [3]:
  auto naive_base64_encode = [&]() {
    base64_encode(buf.data(), buf.size());
  };;

In [4]:
  auto xtl_base64_encode = [&]() {
    std::string buf_as_str(buf.begin(), buf.end());
    xtl::base64encode(buf_as_str);
  };;

benchmark_function is a simple benchmark function see base64_naive.cpp (see below, I compare it to %%timeit; and it gives comparable results)


In [5]:
benchmark_function(naive_base64_encode, 10)

167.277

In [6]:
benchmark_function(xtl_base64_encode, 10)

509.299

In [7]:
%%timeit 
naive_base64_encode();

156 ms +- 5.58 ms per loop (mean +- std. dev. of 7 runs 10 loops each)


The problem is that the execution time could be 30 times faster (debug), or even 80 times faster

````bash
> g++ base64_naive.cpp --std=c++11 -g # debug build
> ./a.out
naive_base64_encode : 6.29823ms

> g++ base64_naive.cpp --std=c++11 -O3 # optimized build
> ./a.out
naive_base64_encode : 2.12909ms
````

This is not due to xeus see cling session below:

````cpp
> cling --std=c++11
Warning in cling::IncrementalParser::CheckABICompatibility():
  Possible C++ standard library mismatch, compiled with __GLIBCXX__ '20170814'
  Extraction of runtime standard library version was: '20180720'

****************** CLING ******************
* Type C++ code and press enter to run it *
*             Type .q to exit             *
*******************************************
[cling]$ .L base64_naive.cpp
[cling]$ auto buf = read_binary_file("data/lena.png");
[cling]$   auto naive_base64_encode = [&]() {
[cling]$ ?       base64_encode(buf.data(), buf.size());
[cling]$ ?     };;
[cling]$
[cling]$ benchmark_function(naive_base64_encode, 10)
(double) 153.873
````